# PREP

In [1]:
import torch
from torchvision import transforms as T
from lightly.transforms import SimCLRTransform, DINOTransform, MAETransform, MoCoV2Transform, utils
from datasets import create_dataset
from models import MAEModel
import pytorch_lightning as pl
import os
import copy
import gc
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader


c:\Users\User\Desktop\studia\Sem 6\WB\Warsztaty_Badawcze\.conda\lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
c:\Users\User\Desktop\studia\Sem 6\WB\Warsztaty_Badawcze\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from pytorch_lightning.callbacks import Callback
import os

class SaveAtEpochsCallback(Callback):
    def __init__(self, save_epochs, dirpath="checkpoints"):
        super().__init__()
        self.save_epochs = set(save_epochs)
        self.dirpath = dirpath
        os.makedirs(self.dirpath, exist_ok=True)

    def on_train_epoch_end(self, trainer, pl_module):
        current_epoch = trainer.current_epoch + 1  # epoka 0-based
        if current_epoch in self.save_epochs:
            filename = f"model_epoch_{current_epoch}.ckpt"
            path = os.path.join(self.dirpath, filename)
            trainer.save_checkpoint(path)
            print(f"Zapisano model po epoce {current_epoch}: {path}")


In [3]:
SEED = 42

def seed_everything(seed: int=42):
    pl.seed_everything(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    ####### Normaly you would also need to seed those generators but `pytorch_lightning` does it in one func
    # random.seed(seed)
    # np.random.seed(seed)
    # torch.manual_seed(seed)
    ######
    torch.cuda.manual_seed(seed) # Don't know if pytorch lightning does this
    torch.cuda.manual_seed_all(seed) # Don't know if pytorch lightning does this
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(SEED)

Seed set to 42


In [4]:
from lightly.transforms import MAETransform

mae_transform = MAETransform(
    input_size=224,
    normalize={
        "mean": utils.IMAGENET_NORMALIZE["mean"],
        "std": utils.IMAGENET_NORMALIZE["std"]
    }
)
# dla supervised treningu
scratch_transform = T.v2.Compose([
    T.RandomResizedCrop((224, 224)),
    T.RandomHorizontalFlip(),
    T.v2.ToImage(),
    T.v2.ToDtype(torch.float32, scale=True),
    T.Normalize(
        mean=utils.IMAGENET_NORMALIZE["mean"],
        std=utils.IMAGENET_NORMALIZE["std"],
    )
])

# dla testu/ewaluacji
test_transform = T.v2.Compose([
    T.Resize((224, 224)),
    T.v2.ToImage(),
    T.v2.ToDtype(torch.float32, scale=True),
    T.Normalize(
        mean=utils.IMAGENET_NORMALIZE["mean"],
        std=utils.IMAGENET_NORMALIZE["std"],
    )
])

c:\Users\User\Desktop\studia\Sem 6\WB\Warsztaty_Badawcze\.conda\lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [5]:
from_path = "./data"
SSL_proportion = 0.9

train_full_dataset_MAE, train_ssl_dataset_MAE, train_dataset_MAE, test_dataset_MAE = create_dataset(
    set_name='CIFAR10',
    SSL_proportion=SSL_proportion,
    train_transform=scratch_transform,              
    train_full_transform=mae_transform.transform,    
    test_transform=test_transform,
    path_to_data=from_path,
    seed=42,
    download=False
)


Length of entire train dataset:  50000
Length of SSL train dataset:  45000
Length of classification train dataset:  5000
Length of test dataset:  10000


In [6]:
BATCH_SIZE = 128
NUM_EPOCHS = 20
LEARNING_RATE = 0.001
NUM_WORKERS = 4
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
#GARBAGE COLLECTOR FAJNA SPRAWA - BEZ NIEGO VRAMu BRAKUJE
if device == "gpu":
    torch.cuda.empty_cache()
    gc.collect()

Using device: cuda


In [7]:
train_loader = DataLoader(train_ssl_dataset_MAE, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, persistent_workers=True)
val_loader = DataLoader(test_dataset_MAE, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, persistent_workers=True)

In [8]:
model1 = MAEModel(
    lr=LEARNING_RATE,
    weight_decay=1e-4,
    max_epochs=50,
    backbone_type="pretrained_resnet18", 
    input_dim=3 * 224 * 224, 
    mask_ratio=0.9
)


c:\Users\User\Desktop\studia\Sem 6\WB\Warsztaty_Badawcze\.conda\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\User\Desktop\studia\Sem 6\WB\Warsztaty_Badawcze\.conda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:

model2 = MAEModel(
    lr=LEARNING_RATE,
    weight_decay=1e-4,
    max_epochs=50,
    backbone_type="random_resnet18", 
    input_dim=3 * 224 * 224, 
    mask_ratio=0.9
)


c:\Users\User\Desktop\studia\Sem 6\WB\Warsztaty_Badawcze\.conda\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
LEARNING_RATE = 0.01

In [10]:

model3 = MAEModel(
    lr=LEARNING_RATE,
    weight_decay=1e-4,
    max_epochs=50,
    backbone_type="pretrained_resnet18", 
    input_dim=3 * 224 * 224, 
    mask_ratio=0.9
)


In [ ]:
model4 = MAEModel(
    lr=LEARNING_RATE,
    weight_decay=1e-4,
    max_epochs=50,
    backbone_type="random_resnet18", 
    input_dim=3 * 224 * 224, 
    mask_ratio=0.9
)

In [ ]:
checkpoint_callback_1 = SaveAtEpochsCallback(save_epochs=[10, 15, 20], dirpath="checkpoints/lr_0.001_mae_cifar10_mask09")

trainer1 = pl.Trainer(
    max_epochs=20,
    callbacks=[checkpoint_callback_1],
    accelerator='auto',
    devices=1,
    log_every_n_steps=10
)


Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:

checkpoint_callback_2 = SaveAtEpochsCallback(save_epochs=[10, 15, 20], dirpath="checkpoints/lr_0.001_mae_cifar10_random_mask09")

trainer2 = pl.Trainer(
    max_epochs=20,
    callbacks=[checkpoint_callback_2],
    accelerator='auto',
    devices=1,
    log_every_n_steps=10
)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:

checkpoint_callback_3 = SaveAtEpochsCallback(save_epochs=[10, 15, 20], dirpath="checkpoints/lr_0.01_mae_cifar10_mask09")

trainer3 = pl.Trainer(
    max_epochs=20,
    callbacks=[checkpoint_callback_3],
    accelerator='auto',
    devices=1,
    log_every_n_steps=10
)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:

checkpoint_callback_4 = SaveAtEpochsCallback(save_epochs=[10, 15, 20], dirpath="checkpoints/lr_0.01_mae_cifar10_random_mask09")

trainer4 = pl.Trainer(
    max_epochs=20,
    callbacks=[checkpoint_callback_4],
    accelerator='auto',
    devices=1,
    log_every_n_steps=10
)

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


# TRAIN

In [ ]:
trainer1.fit(model1, train_dataloaders=train_loader, val_dataloaders=val_loader)
trainer2.fit(model2, train_dataloaders=train_loader, val_dataloaders=val_loader)
trainer3.fit(model3, train_dataloaders=train_loader, val_dataloaders=val_loader)
trainer4.fit(model4, train_dataloaders=train_loader, val_dataloaders=val_loader)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | MAEEncoder | 11.2 M | train
1 | decoder | MAEDecoder | 787 K  | train
-----------------------------------------------
12.0 M    Trainable params
0         Non-trainable params
12.0 M    Total params
47.858    Total estimated model params size (MB)
75        Modules in train mode
0         Modules in eval mode


Epoch 0:   3%|▎         | 12/352 [03:48<1:47:42,  0.05it/s, v_num=5, train_loss=0.0185] 


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined